### Setup and test

In [1]:
# --------------------------------------------------------------------------
# 00_setup_and_test.ipynb
# Spark + Iceberg + MinIO via Polaris REST catalog
# --------------------------------------------------------------------------

from pyspark.sql import SparkSession
from pathlib import Path
import os

# Polaris credentials (are generated into .env by the bootstrap container)
POLARIS_CLIENT_ID = os.getenv("POLARIS_CLIENT_ID")
POLARIS_CLIENT_SECRET = os.getenv("POLARIS_CLIENT_SECRET")

if not POLARIS_CLIENT_ID or not POLARIS_CLIENT_SECRET:
    raise RuntimeError(
        "❌ Polaris credentials ontbreken. Zet POLARIS_CLIENT_ID en POLARIS_CLIENT_SECRET in de omgeving."
    )

# Polaris endpoints + scope
POLARIS_URI = os.getenv("POLARIS_URI", "http://polaris:8181/api/catalog")
POLARIS_OAUTH2 = os.getenv(
    "POLARIS_OAUTH2_TOKEN_URL", "http://polaris:8181/api/catalog/v1/oauth/tokens"
)
POLARIS_SCOPE = os.getenv("POLARIS_SCOPE", "PRINCIPAL_ROLE:ALL")
POLARIS_WAREHOUSE = os.getenv("POLARIS_CATALOG_NAME", "polaris")

# === 0️⃣ Oude Spark-sessie stoppen (veilig) ===
if "spark" in locals():
    try:
        spark.stop()
        print("🧹 Oude Spark-sessie gestopt.")
    except Exception as e:
        print(f"⚠️ Kon Spark niet netjes stoppen: {e}")

# === 1️⃣ SparkSession met Iceberg Polaris REST Catalog ===
spark = (
    SparkSession.builder
        .appName("Lakehouse-Unplugged")

        # Iceberg
        .config(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )

        # Polaris REST catalog
        .config("spark.sql.defaultCatalog", "polaris")
        .config("spark.sql.catalog.polaris", "org.apache.polaris.spark.SparkCatalog")
        .config("spark.sql.catalog.polaris.uri", POLARIS_URI)
        .config("spark.sql.catalog.polaris.warehouse", POLARIS_WAREHOUSE)
        .config(
            "spark.sql.catalog.polaris.credential",
            f"{POLARIS_CLIENT_ID}:{POLARIS_CLIENT_SECRET}"
        )
        .config("spark.sql.catalog.polaris.oauth2-server-uri", POLARIS_OAUTH2)
        .config("spark.sql.catalog.polaris.scope", POLARIS_SCOPE)
        .config(
            "spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation",
            "vended-credentials",
        )
        .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")

        # Performance (licht, PoC)
        .config("spark.sql.shuffle.partitions", "4")
        .config("spark.sql.adaptive.enabled", "true")

        # S3A / MinIO voor de data-files
        .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
        .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
        .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

        .getOrCreate()
)

# === 2️⃣ Validatie ===
hconf = spark._jsc.hadoopConfiguration()

print("✅ Spark gestart via Polaris")
print(f"🔢 Spark versie: {spark.version}")
print(f"📦 Default catalog: {spark.conf.get('spark.sql.defaultCatalog')}")
print(f"🧭 Polaris URI: {spark.conf.get('spark.sql.catalog.polaris.uri')}")
print(f"🏭 Polaris warehouse: {spark.conf.get('spark.sql.catalog.polaris.warehouse')}")
print(f"🎯 Polaris scope: {spark.conf.get('spark.sql.catalog.polaris.scope')}")
print(f"🌐 S3A endpoint: {hconf.get('fs.s3a.endpoint')}")



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/05 07:35:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark gestart via Polaris
🔢 Spark versie: 3.5.1
📦 Default catalog: polaris
🧭 Polaris URI: http://polaris:8181
🏭 Polaris warehouse: polaris
🎯 Polaris scope: PRINCIPAL_ROLE:ALL
🌐 S3A endpoint: http://minio:9000


#### Controleer de setup

In [4]:
from pyspark.sql import SparkSession

print("🔍 Controle: Polaris REST + MinIO via S3A...")

try:
    # Polaris-config uit Spark
    catalog_uri = spark.conf.get("spark.sql.catalog.polaris.uri")
    warehouse = spark.conf.get("spark.sql.catalog.polaris.warehouse")
    scope = spark.conf.get("spark.sql.catalog.polaris.scope")

    print(f"🧭 Polaris URI: {catalog_uri}")
    print(f"🏭 Polaris warehouse: {warehouse}")
    print(f"🎯 Polaris scope: {scope}")

    # Hadoop-config uit Spark voor MinIO I/O
    conf = spark.sparkContext._jsc.hadoopConfiguration()
    endpoint = conf.get("fs.s3a.endpoint")
    impl = conf.get("fs.s3a.impl")
    access_key = conf.get("fs.s3a.access.key")

    print(f"🌐 S3A endpoint: {endpoint}")
    print(f"⚙️  Implementation: {impl}")
    print(f"🔑 Access key: {access_key}")

    print("📡 Probe: probeer S3A pad te controleren...")

    # Test pad: je daadwerkelijke warehouse in MinIO
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark._jvm.java.net.URI("s3a://warehouse/"), conf
    )
    path = spark._jvm.org.apache.hadoop.fs.Path("s3a://warehouse/")

    if fs.exists(path):
        print("✅ Verbinding OK — 'warehouse' bucket is bereikbaar via S3A.")
    else:
        print("⚠️ Verbinding OK, maar 'warehouse' bucket bestaat nog niet.")
        print("   ➜ Spark maakt deze automatisch aan bij de eerste write.")

except Exception as e:
    print("❌ Fout bij verbinding met Polaris/MinIO:")
    print(e)



🔍 Controle: Polaris REST + MinIO via S3A...
🧭 Polaris URI: http://polaris:8181
🏭 Polaris warehouse: polaris
🎯 Polaris scope: PRINCIPAL_ROLE:ALL
🌐 S3A endpoint: http://minio:9000
⚙️  Implementation: org.apache.hadoop.fs.s3a.S3AFileSystem
🔑 Access key: minioadmin
📡 Probe: probeer S3A pad te controleren...


26/01/05 07:36:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


✅ Verbinding OK — 'warehouse' bucket is bereikbaar via S3A.


#### Parkeer bestande in de landingzone

In [5]:
import boto3
from pathlib import Path

# ======================================================================
# 0️⃣ Helper: zoek automatisch lokaal data-bestand
# ======================================================================
def find_data_file(filename: str) -> Path:
    p = Path.cwd()
    for _ in range(4):
        candidate = p / "data" / filename
        if candidate.exists():
            return candidate
        p = p.parent
    raise FileNotFoundError(f"❌ Kon '{filename}' niet vinden in een 'data' map.")

# ======================================================================
# 1️⃣ Config
# ======================================================================
local_file = find_data_file("gekentekendevoertuigen_sample.json")
bucket = "warehouse"
prefix = "landing"

object_key = f"{prefix}/{local_file.name}"
s3_uri = f"s3a://{bucket}/{object_key}"

print(f"📄 Lokaal bestand: {local_file}")
print(f"⬆️ Upload naar:  {s3_uri}")


# ======================================================================
# 2️⃣ MinIO client via boto3
# ======================================================================
s3 = boto3.client(
    "s3",
    endpoint_url="http://minio:9000",
    aws_access_key_id="minioadmin",
    aws_secret_access_key="minioadmin",
    region_name="us-east-1",
)

# Upload bestand
s3.upload_file(str(local_file), bucket, object_key)

print("✅ Upload gelukt.")


# ======================================================================
# 3️⃣ Verify: lijst objecten in prefix
# ======================================================================
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
print("📦 Objecten in MinIO:")
for item in response.get("Contents", []):
    print(" -", item["Key"])


# ======================================================================
# 4️⃣ Spark read via S3A (data-files), metadata via Polaris
# ======================================================================
print(f"📥 Inlezen via Spark: {s3_uri}")

df = spark.read.option("multiline", "true").json(s3_uri)

print(f"📊 Aantal records: {df.count():,}")
df.printSchema()



📄 Lokaal bestand: /workspace/data/gekentekendevoertuigen_sample.json
⬆️ Upload naar:  s3a://warehouse/landing/gekentekendevoertuigen_sample.json
✅ Upload gelukt.
📦 Objecten in MinIO:
 - landing/gekentekendevoertuigen_sample.json
📥 Inlezen via Spark: s3a://warehouse/landing/gekentekendevoertuigen_sample.json


Py4JJavaError: An error occurred while calling o69.count.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'polaris': org.apache.polaris.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.CatalogManager.currentCatalog(CatalogManager.scala:122)
	at org.apache.spark.sql.connector.catalog.CatalogManager.currentNamespace(CatalogManager.scala:93)
	at org.apache.spark.sql.catalyst.optimizer.ReplaceCurrentLike.apply(finishAnalysis.scala:143)
	at org.apache.spark.sql.catalyst.optimizer.Repl

#### Ingest into bronze table

In [6]:
# ======================================================================
# 🔄 Ingest van Landingzone → Bronze (Iceberg via Polaris)
# ======================================================================

# Input configuratie (aangeleverd vanuit eerdere cel)
bucket = "warehouse"
prefix = "landing"
local_file = find_data_file("gekentekendevoertuigen_sample.json")   # naam komt uit je workflow
object_key = f"{prefix}/{local_file.name}"
s3_uri = f"s3a://{bucket}/{object_key}"

bronze_table = "polaris.bronze.gekentekendevoertuigen"

print(f"📥 Lezen vanuit landingzone: {s3_uri}")

# 1️⃣ Data inlezen uit landingzone
df = spark.read.json(s3_uri)

print(f"📦 Aantal records geladen: {df.count()}")

# 2️⃣ Namespace garanderen
spark.sql("CREATE NAMESPACE IF NOT EXISTS polaris.bronze")

# 3️⃣ Wegschrijven naar Iceberg Bronze
print(f"🧊 Schrijven naar Bronze tabel: {bronze_table}")

(
    df.writeTo(bronze_table)
      .using("iceberg")
      .option("format-version", "2")
      .option("overwrite-mode", "dynamic")
      .createOrReplace()
)

print(f"✅ Bronze tabel bijgewerkt: {bronze_table}")

# 4️⃣ Tabellen tonen
print("
📋 Tabellen in polaris.bronze:")
spark.sql("SHOW TABLES IN polaris.bronze").show(truncate=False)

# 5️⃣ Bronze teruglezen ter controle
bronze_df = spark.read.table(bronze_table)

print(f"
🔁 Records in Bronze: {bronze_df.count():,}")
bronze_df.show(5, truncate=False)



📥 Lezen vanuit landingzone: s3a://warehouse/landing/gekentekendevoertuigen_sample.json


📦 Aantal records geladen: 10002
🧊 Schrijven naar Bronze tabel: lakehouse.bronze.gekentekendevoertuigen


25/11/22 10:28:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/22 10:28:55 WARN HadoopTableOperations: Error reading version hint file s3a://warehouse/bronze/gekentekendevoertuigen/metadata/version-hint.text
java.io.FileNotFoundException: No such file or directory: s3a://warehouse/bronze/gekentekendevoertuigen/metadata/version-hint.text
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3866)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.extractOrFetchSimpleFileStatus(S3AFileSystem.java:5401)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:1465)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:1441)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.iceberg.hadoop.HadoopTableOperations

✅ Bronze tabel bijgewerkt: lakehouse.bronze.gekentekendevoertuigen

📋 Tabellen in lakehouse.bronze:
+---------+----------------------+-----------+
|namespace|tableName             |isTemporary|
+---------+----------------------+-----------+
|bronze   |gekentekendevoertuigen|false      |
+---------+----------------------+-----------+


🔁 Records in Bronze: 10,002
+---------------+----------------------------+----------------------------+----------------+-------------+-----------------------+--------------------+-------------+------------------+-----------------------------------------------+---------------------------------------------+---------------------------+-----------------------------------------------+-----------------------------------------------+-----------------------------------------------+-------------------------------------------------+-----------------------------------------------+-------+------------------------+------------------------+---------+--------------+----

#### Query de bronze table

In [22]:
from pyspark.sql.functions import col

df = spark.read.table("polaris.bronze.gekentekendevoertuigen")

print("🚗 Top 5 voertuigsoorten:")
(
    df.groupBy("voertuigsoort")
      .count()
      .orderBy(col("count").desc())
      .show(5, truncate=False)
)

print("
🏷️ Top 5 merken:")
(
    df.groupBy("merk")
      .count()
      .orderBy(col("count").desc())
      .show(5, truncate=False)
)

print("
🔤 Top 5 handelsbenamingen:")
(
    df.groupBy("handelsbenaming")
      .count()
      .orderBy(col("count").desc())
      .show(5, truncate=False)
)

print("
⚡ Top 5 voertuigen op vermogen (massarijklaar):")
(
    df.select("merk", "handelsbenaming", "vermogen_massarijklaar")
      .orderBy(col("vermogen_massarijklaar").desc_nulls_last())
      .show(5, truncate=False)
)



🚗 Top 5 voertuigsoorten:
+--------------------+-----+
|voertuigsoort       |count|
+--------------------+-----+
|Personenauto        |7078 |
|Bedrijfsauto        |1237 |
|Bromfiets           |782  |
|Motorfiets          |258  |
|Middenasaanhangwagen|136  |
+--------------------+-----+
only showing top 5 rows


🏷️ Top 5 merken:
+-------------+-----+
|merk         |count|
+-------------+-----+
|VOLKSWAGEN   |1076 |
|PEUGEOT      |615  |
|RENAULT      |606  |
|MERCEDES-BENZ|565  |
|FORD         |553  |
+-------------+-----+
only showing top 5 rows


🔤 Top 5 handelsbenamingen:
+---------------+-----+
|handelsbenaming|count|
+---------------+-----+
|POLO           |219  |
|GOLF           |202  |
|FOCUS          |138  |
|N/A            |135  |
|CLIO           |125  |
+---------------+-----+
only showing top 5 rows


⚡ Top 5 voertuigen op vermogen (massarijklaar):
+-------+--------------------+----------------------+
|merk   |handelsbenaming     |vermogen_massarijklaar|
+-------+-------------